<a href="https://colab.research.google.com/github/Aaron3013352/customer-churn-prediction/blob/main/notebooks/01_sql_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL-based Feature Extraction (SQLite)

This notebook simulates a production-style data workflow using SQL.
Raw Telco churn data is loaded into a SQLite database, queried for
business insights, and transformed into a clean, model-ready dataset.


In [1]:
import pandas as pd
import sqlite3
from pathlib import Path


In [2]:
RAW_DATA_PATH = "/content/sample_data/Telco_customer_churn.xlsx"

df_raw = pd.read_excel(RAW_DATA_PATH)
df_raw.head()


,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [3]:
# Create processed folder
Path("data/processed").mkdir(parents=True, exist_ok=True)

DB_PATH = "data/processed/churn.db"
con = sqlite3.connect(DB_PATH)

# Write raw data to SQL table
df_raw.to_sql("telco_churn_raw", con, if_exists="replace", index=False)

# Sanity check
pd.read_sql_query(
    "SELECT COUNT(*) AS n_rows FROM telco_churn_raw;",
    con
)


,n_rows
0,7043


In [4]:
def run_sql(query: str) -> pd.DataFrame:
    return pd.read_sql_query(query, con)


In [5]:
run_sql("""
    SELECT
        "Churn Label" AS churn_label,
        COUNT(*) AS customers
    FROM telco_churn_raw
    GROUP BY "Churn Label";
""")


,churn_label,customers
0,No,5174
1,Yes,1869


In [6]:
run_sql("""
    SELECT
        Contract,
        COUNT(*) AS total_customers,
        SUM(CASE WHEN "Churn Label" = 'Yes' THEN 1 ELSE 0 END) AS churners,
        ROUND(
            1.0 * SUM(CASE WHEN "Churn Label" = 'Yes' THEN 1 ELSE 0 END) / COUNT(*),
            3
        ) AS churn_rate
    FROM telco_churn_raw
    GROUP BY Contract
    ORDER BY churn_rate DESC;
""")


,Contract,total_customers,churners,churn_rate
0,Month-to-month,3875,1655,0.427
1,One year,1473,166,0.113
2,Two year,1695,48,0.028


In [7]:
run_sql("""
    SELECT
        "Internet Service" AS internet_service,
        COUNT(*) AS total_customers,
        SUM(CASE WHEN "Churn Label" = 'Yes' THEN 1 ELSE 0 END) AS churners,
        ROUND(
            1.0 * SUM(CASE WHEN "Churn Label" = 'Yes' THEN 1 ELSE 0 END) / COUNT(*),
            3
        ) AS churn_rate
    FROM telco_churn_raw
    GROUP BY "Internet Service"
    ORDER BY churn_rate DESC;
""")


,internet_service,total_customers,churners,churn_rate
0,Fiber optic,3096,1297,0.419
1,DSL,2421,459,0.190
2,No,1526,113,0.074


In [8]:
run_sql("""
    SELECT
        "Churn Label" AS churn_label,
        ROUND(AVG("Tenure Months"), 1) AS avg_tenure,
        ROUND(AVG("Monthly Charges"), 2) AS avg_monthly_charges,
        ROUND(AVG("Total Charges"), 2) AS avg_total_charges
    FROM telco_churn_raw
    GROUP BY "Churn Label";
""")


,churn_label,avg_tenure,avg_monthly_charges,avg_total_charges
0,No,37.6,61.27,2549.91
1,Yes,18.0,74.44,1531.80


In [9]:
con.execute("DROP VIEW IF EXISTS churn_features;")

con.execute("""
    CREATE VIEW churn_features AS
    SELECT
        -- target
        "Churn Label" AS churn_label,

        -- numeric features
        "Count",
        "Tenure Months",
        "Monthly Charges",
        "Total Charges",
        "CLTV",

        -- categorical features
        "Gender",
        "Senior Citizen",
        "Partner",
        "Dependents",
        "Phone Service",
        "Multiple Lines",
        "Internet Service",
        "Online Security",
        "Online Backup",
        "Device Protection",
        "Tech Support",
        "Streaming TV",
        "Streaming Movies",
        "Contract",
        "Paperless Billing",
        "Payment Method"

    FROM telco_churn_raw;
""")


In [10]:
run_sql("SELECT * FROM churn_features LIMIT 5;")


,churn_label,Count,Tenure Months,Monthly Charges,Total Charges,CLTV,Gender,Senior Citizen,Partner,Dependents,...,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method
0,Yes,1,2,53.85,108.15,3239,Male,No,No,No,...,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check
1,Yes,1,2,70.70,151.65,2701,Female,No,No,Yes,...,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check
2,Yes,1,8,99.65,820.5,5372,Female,No,No,Yes,...,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check
3,Yes,1,28,104.80,3046.05,5003,Female,No,Yes,Yes,...,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check
4,Yes,1,49,103.70,5036.3,5340,Male,No,No,Yes,...,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic)


In [11]:
df_features = run_sql("SELECT * FROM churn_features;")
df_features.shape


(7043, 22)

In [12]:
# Save to processed folder
df_features.to_csv("data/processed/churn_features.csv", index=False)
df_features.to_parquet("data/processed/churn_features.parquet", index=False)

df_features.head()


,churn_label,Count,Tenure Months,Monthly Charges,Total Charges,CLTV,Gender,Senior Citizen,Partner,Dependents,...,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method
0,Yes,1,2,53.85,108.15,3239,Male,No,No,No,...,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check
1,Yes,1,2,70.70,151.65,2701,Female,No,No,Yes,...,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check
2,Yes,1,8,99.65,820.5,5372,Female,No,No,Yes,...,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check
3,Yes,1,28,104.80,3046.05,5003,Female,No,Yes,Yes,...,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check
4,Yes,1,49,103.70,5036.3,5340,Male,No,No,Yes,...,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic)


In [13]:
con.close()


## Output

The SQL-extracted feature table is saved under:

- `data/processed/churn_features.csv`
- `data/processed/churn_features.parquet`

These files are used as inputs for:
- `02_eda.ipynb`
- `03_modeling.ipynb`
